In [1]:
import pandas as pd
import numpy as np

First, data from wikipedia were prepared outside of python, then loaded as xlsx.

In [2]:
dfToronto1 = pd.read_excel('C:/Users/LuciaBenisova/Toronto1.xlsx')
dfToronto1.head()

,Postcode,Borough,Neigborhood,Latitude,Longitude
0,M1G,Scarborough,Woburn,43.770992,-79.216917
1,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
3,M1S,Scarborough,Agincourt,43.794200,-79.262029
4,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389


Then, Borough containing "toronto" has been flagged,m for further analysis

In [3]:
dfToronto1['TorontoFlag'] = np.where(dfToronto1['Borough'].str.contains("Toronto", case=False, na=False), 'Y', '')
dfToronto1

,Postcode,Borough,Neigborhood,Latitude,Longitude,TorontoFlag
0,M1G,Scarborough,Woburn,43.770992,-79.216917,
1,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,
2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,
3,M1S,Scarborough,Agincourt,43.794200,-79.262029,
4,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,
5,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,
7,M2K,North York,Bayview Village,43.786947,-79.385975,
8,M2N,North York,Willowdale South,43.770120,-79.408493,
9,M2P,North York,York Mills West,43.752758,-79.400049,


In [4]:
from geopy.geocoders import Nominatim

Check, the new dataset

In [36]:
toronto_borough_data = dfToronto1[dfToronto1['TorontoFlag'] == 'Y'].reset_index(drop=True)
toronto_borough_data.head()

,Postcode,Borough,Neigborhood,Latitude,Longitude,TorontoFlag
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Y
1,M4M,East Toronto,Studio District,43.659526,-79.340923,Y
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,Y
3,M4P,Central Toronto,Davisville North,43.712751,-79.390197,Y
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,Y


Below, I am going to try to visualize the Borough.

In [37]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [39]:
import folium

In [41]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_borough_data['Latitude'], toronto_borough_data['Longitude'], toronto_borough_data['Neigborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

I am going to work with Foursquare, using thei API to check venuew within all Boroughs containing text string "Toronto"

In [9]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [10]:
CLIENT_ID = 'G0UDA5V0QKQYTPYWMVYVUKDY02BHOFJ13WCR415TFJZSSBHR' # your Foursquare ID
CLIENT_SECRET = 'HIQOJPCSUFQA0YU50PZCN5J0DS21E3IRXFHF2AX1C4B0GG4B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G0UDA5V0QKQYTPYWMVYVUKDY02BHOFJ13WCR415TFJZSSBHR
CLIENT_SECRET:HIQOJPCSUFQA0YU50PZCN5J0DS21E3IRXFHF2AX1C4B0GG4B


In [42]:
toronto_borough_data.loc[5, 'Neigborhood']

'Davisville'

In [43]:
neighborhood_latitude = toronto_borough_data.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_borough_data.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_borough_data.loc[5, 'Neigborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Davisville are 43.7043244, -79.3887901.


In [44]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=G0UDA5V0QKQYTPYWMVYVUKDY02BHOFJ13WCR415TFJZSSBHR&client_secret=HIQOJPCSUFQA0YU50PZCN5J0DS21E3IRXFHF2AX1C4B0GG4B&v=20180605&ll=43.7043244,-79.3887901&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb5cbb59fb6b776fcaf4252'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Rouge',
  'headerFullLocation': 'Rouge, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.8406247045, 'lng': -79.19940921111892},
   'sw': {'lat': 43.8316246955, 'lng': -79.2118629888811}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
print('done')

done


In [47]:
toronto_venues = getNearbyVenues(names=toronto_borough_data['Neigborhood'],
                                   latitudes=dfToronto1['Latitude'],
                                   longitudes=dfToronto1['Longitude']
                                  )


The Beaches
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Rosedale
Church and Wellesley
St. James Town
Berczy Park
Central Bay Street
Roselawn
Stn A PO Boxes 25 The Esplanade
Christie
Business Reply Mail Processing Centre 969 Eastern
The Danforth West, Riverdale
The Beaches West, India Bazaar
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Cabbagetown, St. James Town
Harbourfront, Regent Park
Ryerson, Garden District
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
First Canadian Place, Underground city
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, 

In [50]:
print(toronto_borough_data.shape)
toronto_borough_data.head()

(38, 6)


,Postcode,Borough,Neigborhood,Latitude,Longitude,TorontoFlag
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Y
1,M4M,East Toronto,Studio District,43.659526,-79.340923,Y
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,Y
3,M4P,Central Toronto,Davisville North,43.712751,-79.390197,Y
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,Y


In [51]:
toronto_borough_data.groupby('Borough').count()

,Postcode,Neigborhood,Latitude,Longitude,TorontoFlag
Borough,,,,,
Central Toronto,9,9,9,9,9
Downtown Toronto,18,18,18,18,18
East Toronto,5,5,5,5,5
West Toronto,6,6,6,6,6


Now I am going to cluster the selected neighborhoods.

In [54]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Done')

Done


Column called "toronto"flag" must be removed before clustring the data

In [63]:
toronto_borough_data1= toronto_borough_data.drop(["TorontoFlag","Postcode","Borough","Neigborhood"], axis=1)
toronto_borough_data1.head()

,Latitude,Longitude
0,43.676357,-79.293031
1,43.659526,-79.340923
2,43.728020,-79.388790
3,43.712751,-79.390197
4,43.715383,-79.405678


In [76]:
clusters = 3

k_means = KMeans(init="k-means++", n_clusters=clusters, n_init=12)
k_means.fit(toronto_borough_data1)
labels = k_means.labels_

print(labels)

[0 0 1 1 1 1 0 0 0 0 0 1 0 2 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 2 2 2 2 2
 2]


In [67]:
toronto_borough_data1["Labels"] = labels
toronto_borough_data1

,Latitude,Longitude,Labels
0,43.676357,-79.293031,2
1,43.659526,-79.340923,2
2,43.728020,-79.388790,0
3,43.712751,-79.390197,0
4,43.715383,-79.405678,0
5,43.704324,-79.388790,0
6,43.679563,-79.377529,2
7,43.665860,-79.383160,2
8,43.651494,-79.375418,2
9,43.644771,-79.373306,2


In [68]:
toronto_borough_data1.groupby('Labels').mean()

,Latitude,Longitude
Labels,,
0,43.701980,-79.398954
1,43.655066,-79.445472
2,43.656940,-79.368485
